In [ ]:
read.json("path")

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window
import os

from google.cloud import storage
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="sca-at-b5179177a346.json"

spark = (SparkSession.builder
        .master("local")
        .appName("projetosoulcode") 
        .getOrCreate())



In [21]:
schema = StructType(
   [StructField('concessionaria', StringType(), True),
    StructField('ano_do_pnv_snv', StringType(), True),
    StructField('tipo_de_rodovia', StringType(), True),
    StructField('rodovia', StringType(), True),
    StructField('uf', StringType(), True),
    StructField('km_m_inicial', StringType(), True),
    StructField('km_m_final', StringType(), True),
    StructField('direcao', StringType(), True),
    StructField('sentido', StringType(), True),
   ]
  )

In [13]:
df = spark.read.option("delimiter", ";").option("encoding", "ISO-8859-1").csv("dados_dos_trechos_concedidos (3).csv", header=True)
df.printSchema()
df.show()

root
 |-- concessionaria: string (nullable = true)
 |-- ano_do_pnv_snv: string (nullable = true)
 |-- tipo_de_rodovia: string (nullable = true)
 |-- rodovia: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- km_m_inicial: string (nullable = true)
 |-- km_m_final: string (nullable = true)
 |-- direcao: string (nullable = true)
 |-- sentido: string (nullable = true)

+--------------------+--------------+---------------+--------+---+------------+----------+-------+-----------+
|      concessionaria|ano_do_pnv_snv|tipo_de_rodovia| rodovia| uf|km_m_inicial|km_m_final|direcao|    sentido|
+--------------------+--------------+---------------+--------+---+------------+----------+-------+-----------+
|AUTOPISTA FERNÃO ...|          2007|         Outros|Contorno| MG|       0,000|     7,400|    Sul|  Crescente|
|AUTOPISTA FERNÃO ...|          2007|         Outros|Contorno| MG|       7,400|     0,000|  Norte|Decrescente|
|AUTOPISTA FERNÃO ...|          2007|         Outros|  Alça 1| 

In [14]:
df = df.withColumn('km_m_inicial', regexp_replace('km_m_inicial', ',', '.').cast('float'))
df = df.withColumn('km_m_final', regexp_replace('km_m_final', ',', '.').cast('float'))

In [15]:
df=df.filter(df.tipo_de_rodovia=="Federal")

In [16]:
df.count()

72

In [17]:
df.show()

+--------------------+--------------+---------------+-------+---+------------+----------+-------+-----------+
|      concessionaria|ano_do_pnv_snv|tipo_de_rodovia|rodovia| uf|km_m_inicial|km_m_final|direcao|    sentido|
+--------------------+--------------+---------------+-------+---+------------+----------+-------+-----------+
|AUTOPISTA FLUMINENSE|          2019|        Federal| BR-101| RJ|         0.0|     322.1|    Sul|  Crescente|
|AUTOPISTA FLUMINENSE|          2019|        Federal| BR-101| RJ|       322.1|       0.0|  Norte|Decrescente|
|AUTOPISTA LITORAL...|          2009|        Federal| BR-116| PR|        71.1|     115.2|  Oeste|  Crescente|
|AUTOPISTA LITORAL...|          2009|        Federal| BR-116| PR|        71.1|     115.2|  Leste|Decrescente|
|AUTOPISTA LITORAL...|          2009|        Federal| BR-101| SC|         0.0|    244.68|    Sul|  Crescente|
|AUTOPISTA LITORAL...|          2009|        Federal| BR-101| SC|         0.0|    244.68|  Norte|Decrescente|
|AUTOPISTA

In [20]:
df = df.withColumn('rodovia', regexp_replace('rodovia', 'BR-', ' '))

In [22]:
df.show()

+--------------------+--------------+---------------+-------+---+------------+----------+-------+-----------+
|      concessionaria|ano_do_pnv_snv|tipo_de_rodovia|rodovia| uf|km_m_inicial|km_m_final|direcao|    sentido|
+--------------------+--------------+---------------+-------+---+------------+----------+-------+-----------+
|AUTOPISTA FLUMINENSE|          2019|        Federal|    101| RJ|         0.0|     322.1|    Sul|  Crescente|
|AUTOPISTA FLUMINENSE|          2019|        Federal|    101| RJ|       322.1|       0.0|  Norte|Decrescente|
|AUTOPISTA LITORAL...|          2009|        Federal|    116| PR|        71.1|     115.2|  Oeste|  Crescente|
|AUTOPISTA LITORAL...|          2009|        Federal|    116| PR|        71.1|     115.2|  Leste|Decrescente|
|AUTOPISTA LITORAL...|          2009|        Federal|    101| SC|         0.0|    244.68|    Sul|  Crescente|
|AUTOPISTA LITORAL...|          2009|        Federal|    101| SC|         0.0|    244.68|  Norte|Decrescente|
|AUTOPISTA

In [24]:
df=df.withColumnRenamed('rodovia', 'br')

In [26]:
df.show(72)

+--------------------+--------------+---------------+----+---+------------+----------+-------+-----------+
|      concessionaria|ano_do_pnv_snv|tipo_de_rodovia|  br| uf|km_m_inicial|km_m_final|direcao|    sentido|
+--------------------+--------------+---------------+----+---+------------+----------+-------+-----------+
|AUTOPISTA FLUMINENSE|          2019|        Federal| 101| RJ|         0.0|     322.1|    Sul|  Crescente|
|AUTOPISTA FLUMINENSE|          2019|        Federal| 101| RJ|       322.1|       0.0|  Norte|Decrescente|
|AUTOPISTA LITORAL...|          2009|        Federal| 116| PR|        71.1|     115.2|  Oeste|  Crescente|
|AUTOPISTA LITORAL...|          2009|        Federal| 116| PR|        71.1|     115.2|  Leste|Decrescente|
|AUTOPISTA LITORAL...|          2009|        Federal| 101| SC|         0.0|    244.68|    Sul|  Crescente|
|AUTOPISTA LITORAL...|          2009|        Federal| 101| SC|         0.0|    244.68|  Norte|Decrescente|
|AUTOPISTA PLANALT...|          2017|